# Process Metrics For All Native Exchange Metabolites #

### File to generate Fig 4 and Fig S8 ###

In [2]:
import numpy as np
import cameo
import pandas as pd
from copy import deepcopy
import pickle
import time
import multiprocessing
from joblib import Parallel, delayed
from mcpecaso.plotting import multiplot_envelopes,plot_envelope,plot_pecaso_dfba,\
                              multi_two_stage_char_contours,two_stage_char_contour
from mcpecaso.core import mcPECASO
import mcpecaso as mcpecaso

mcpecaso.settings.time_end = 100
mcpecaso.settings.initial_biomass = 0.05
mcpecaso.settings.initial_substrate = 500
mcpecaso.settings.num_timepoints = 10000
mcpecaso.settings.scope = 'extrema'

num_cores = multiprocessing.cpu_count()
model = cameo.models.bigg.iJO1366

In [3]:
metabolite_names_dict = {'Spermidine': 'Spermidine',
                         'L-Tryptophan': 'L-Tryptophan',
                         'Thymidine C10H14N2O5': 'Thymidine',
                         'L-Phenylalanine': 'L-Phenylalanine',
                         'Adenosine': 'Adenosine',
                         'Indole': 'Indole',
                         'L-Tyrosine': 'L-Tyrosine',
                         'Inosine': 'Inosine',
                         'Xanthosine': 'Xanthosine',
                         'Cytidine': 'Cytidine',
                         'Uridine': 'Uridine',
                         'Quinate': 'Quinate',
                         'L-Isoleucine': 'L-Isoleucine',
                         'L-Lysine': 'L-Lysine',
                         'Hexanoate (n-C6:0)': 'Hexanoate',
                         'L-Leucine': 'L-Leucine',
                         'L-Arginine': 'L-Arginine',
                         'L-Histidine': 'L-Histidine',
                         'D-Gluconate': 'D-Gluconic Acid',
                         'L-Idonate': 'L-Idonic Acid',
                         '5-Dehydro-D-gluconate': '5-Ketogluconate',
                         'Citrate': 'Citric Acid',
                         'Agmatine': 'Agmatine',
                         'Ornithine': 'Ornithine',
                         'L-Proline': 'L-Proline',
                         'L-Valine': 'L-Valine',
                         'Thymine C5H6N2O2': 'Thymine',
                         'Adenine': 'Adenine',
                         'Guanine': 'Guanine',
                         'Hypoxanthine': 'Hypoxanthine',
                         'O-Acetyl-L-serine': 'O-Acetyl-L-serine',
                         'Xanthine': 'Xanthine',
                         'L-Glutamate': 'L-Glutamate',
                         '2-Oxoglutarate': '\u03b1-Ketoglutarate',
                         'Putrescine': 'Putrescine',
                         'L-Threonine': 'L-Threonine',
                         '4-Aminobutanoate': '\u03b3-Aminobutyrate',
                         'L-Homoserine': 'L-Homoserine',
                         'Allantoin': 'Allantoin',
                         'Succinate': 'Succinate',
                         'Uracil': 'Uracil',
                         'L-Asparagine': 'L-Asparagine',
                         'L-Malate': 'L-Malate',
                         'L-Aspartate': 'L-Aspartate',
                         'L-Cysteine': 'L-Cysteine',
                         'Glycerol 3-phosphate': 'Glycerol3-phosphate',
                         '3-Hydroxypropanoate': '3-Hydroxypropanoate',
                         '(S)-Propane-1,2-diol': '(S)-Propanediol',
                         '(R)-Propane-1,2-diol': '(R)-Propanediol',
                         'D-Alanine': 'D-Alanine',
                         'Glycerol': 'Glycerol',
                         '(R)-Glycerate': '(R)-Glycerate',
                         'D-Glyceraldehyde': 'Glyceraldehyde',
                         'L-Lactate': 'L-Lactate',
                         'Dihydroxyacetone': 'Dihydroxyacetone',
                         'L-Alanine': 'L-Alanine',
                         'D-Lactate': 'D-Lactate',
                         'L-Serine': 'L-Serine',
                         'Pyruvate': 'Pyruvate',
                         'Ethanolamine': 'Ethanolamine',
                         'Ethanol': 'Ethanol',
                         'Acetaldehyde': 'Acetaldehyde',
                         'Glycine': 'Glycine',
                         'Acetate': 'Acetate',
                         'Glycolate C2H3O3': 'Glycolate',
                         'Urea CH4N2O': 'Urea',
                         'Formate': 'Formate',
                         'Reduced glutathione': 'Glutathione',
                         '5-Methylthio-D-ribose': '5-Methylthioribose',
                         '1,5-Diaminopentane': 'Cadaverine'}

metabolite_lookup_dict = {metabolite_names_dict[key]:key for key in metabolite_names_dict}

carbon_dict = {metabolite.name:str(metabolite.elements['C'])
               if 'C' in metabolite.elements else '0'
               for metabolite in model.metabolites}

carbon_dict = {metabolite:carbon_dict[metabolite] 
               if int(carbon_dict[metabolite])<=6 else '>6'
               for metabolite in metabolite_names_dict}

carbon_dict = {metabolite_names_dict[metabolite]:carbon_dict[metabolite] for metabolite in carbon_dict}

data = {'names':[product for product in carbon_dict.keys()], 'number':list(carbon_dict.values())}
metabolite_df = pd.DataFrame(data,index=list(carbon_dict.keys()))
metabolite_order = metabolite_df.sort_values(by=['number','names']).names.values

In [4]:
import sys
import colorlover as cl
from plotly import tools, subplots
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"
import os

try:
    _ = __IPYTHON__
except NameError:
    from plotly.offline import plot
else:
    if 'ipykernel' in sys.modules:
        from plotly.offline import init_notebook_mode
        from plotly.offline import iplot as plot
        from IPython.display import HTML
        HTML("""
             <script>
              var waitForPlotly = setInterval( function() {
              if( typeof(window.Plotly) !== "undefined" ){
              MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
              MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
              clearInterval(waitForPlotly);}}, 250 );
            </script>
            """
        )
        init_notebook_mode(connected=True)
    elif 'IPython' in sys.modules:
        from plotly.offline import plot
    else:
        warn('Unknown ipython configuration')
        from plotly.offline import plot


In [ ]:
def pecaso_calculate(model, product, settings):
    
    
    pecaso_model = deepcopy(model)
    
    target_rxn = [reaction for reaction in pecaso_model.exchanges
                  if reaction.reactants[0].name==product][0]
    pecaso = mcPECASO(model=pecaso_model,biomass_rxn=pecaso_model.reactions.BIOMASS_Ec_iJO1366_core_53p95M,
                            substrate_rxn=pecaso_model.reactions.EX_glc__D_e, target_rxn=target_rxn,
                            condition='Product - '+str(metabolite_names_dict[product]))
    pecaso.settings = settings
    pecaso.calculate_fermentation_characteristics()

    return pecaso


def pecaso_calculate_yield_constraint(model, product, settings, yield_constraint):
    
    
    pecaso_model = deepcopy(model)
    
    target_rxn = [reaction for reaction in pecaso_model.exchanges
                  if reaction.reactants[0].name==product][0]
    pecaso = mcPECASO(model=pecaso_model,biomass_rxn=pecaso_model.reactions.BIOMASS_Ec_iJO1366_core_53p95M,
                            substrate_rxn=pecaso_model.reactions.EX_glc__D_e, target_rxn=target_rxn,
                            condition='Product - '+str(metabolite_names_dict[product]))
    
    pecaso.settings = settings
    pecaso.settings.yield_constraint = yield_constraint * max(pecaso.production_envelope.yield_ub)
    pecaso.calculate_fermentation_characteristics()

    return pecaso


def pecaso_calculate_productivity_constraint(model, product, settings, productivity_constraint):
    pecaso_model = deepcopy(model)
    
    target_rxn = [reaction for reaction in pecaso_model.exchanges
                  if reaction.reactants[0].name==product][0]
    pecaso = mcPECASO(model=pecaso_model,biomass_rxn=pecaso_model.reactions.BIOMASS_Ec_iJO1366_core_53p95M,
                            substrate_rxn=pecaso_model.reactions.EX_glc__D_e, target_rxn=target_rxn,
                            condition='Product - '+str(metabolite_names_dict[product]))
    
    pecaso.settings = settings
    pecaso.settings.productivity_constraint = productivity_constraint/target_rxn.reactants[0].formula_weight*1000
    pecaso.calculate_fermentation_characteristics()

    return pecaso

mcpecaso.settings.uptake_fun = 'logistic'
mcpecaso.settings.uptake_params = {'B': 5}
mcpecaso.settings.objective = 'batch_productivity'
mcpecaso.settings.initial_substrate = 500
pecaso_dict = {}
pecaso_list = []


new_pecaso_list = Parallel(n_jobs=num_cores, verbose=5)(delayed(pecaso_calculate)(model, product,
                                                                                 mcpecaso.settings)
                                                        for product in metabolite_names_dict.keys())

pecaso_dict = {metabolite_names_dict[pecaso_object.target_rxn.reactants[0].name]: pecaso_object 
               for pecaso_object in new_pecaso_list}

pickling_on = open("pecaso_obj_prod.pickle","wb")
pickle.dump(pecaso_dict, pickling_on)
pickling_on.close()

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


# Import and process precalculated fermentation metric data #

In [6]:
pickling_off = open("pecaso_obj_prod.pickle","rb")
pecaso_dict_prod = pickle.load(pickling_off)
pickling_off.close()

pickling_off = open("pecaso_obj_yield_prod_constrained_2_native_mets.pickle","rb")
pecaso_dict_yield_2_prod = pickle.load(pickling_off)
pickling_off.close()

pickling_off = open("pecaso_obj_prod_yield_constrained_75_native_mets.pickle","rb")
pecaso_dict_prod_75_yield = pickle.load(pickling_off)
pickling_off.close()


In [5]:
pecaso_dict_yield_2_prod['D-Lactate'].one_stage_best_batch.batch_yield

1.2050163128924472

In [9]:
metrics = ['batch_productivity','batch_yield','batch_titer']
pecaso_dict = pecaso_dict_prod

pecaso_dicts = {'prod':pecaso_dict_prod,
                'yield_2_prod':pecaso_dict_yield_2_prod,
                'prod_75_yield':pecaso_dict_prod_75_yield}

process_metrics = {'prod':{},
                   'yield_2_prod':{},
                   'prod_75_yield':{}}

products = [product for product in list(pecaso_dict.keys())]

for objective in pecaso_dicts.keys():
    pecaso_dict = pecaso_dicts[objective]
    for i, metric in enumerate(metrics):
        if metric in ['batch_productivity', 'batch_titer']:
            ts_best = [getattr(pecaso_dict[product].two_stage_best_batch, metric)*\
                       pecaso_dict[product].target_rxn.reactants[0].formula_weight/1000 
                       if pecaso_dict[product].two_stage_best_batch else np.nan
                       for  product in list(pecaso_dict.keys())]
            os_best = [getattr(pecaso_dict[product].one_stage_best_batch, metric)*\
                       pecaso_dict[product].target_rxn.reactants[0].formula_weight/1000 
                       if pecaso_dict[product].one_stage_best_batch else np.nan
                       for  product in list(pecaso_dict.keys())]
            ts_sub = [getattr(pecaso_dict[product].two_stage_suboptimal_batch, metric)*\
                      pecaso_dict[product].target_rxn.reactants[0].formula_weight/1000 
                      if pecaso_dict[product].two_stage_suboptimal_batch else np.nan
                      for  product in list(pecaso_dict.keys())]  

        if metric=='batch_yield':
            ts_best = [getattr(pecaso_dict[product].two_stage_best_batch, metric)/\
                       max(pecaso_dict[product].production_envelope.yield_ub)*100 
                       if pecaso_dict[product].two_stage_best_batch else np.nan
                       for  product in list(pecaso_dict.keys())]
            os_best = [getattr(pecaso_dict[product].one_stage_best_batch, metric)/\
                       max(pecaso_dict[product].production_envelope.yield_ub)*100 
                       if pecaso_dict[product].one_stage_best_batch else np.nan
                       for  product in list(pecaso_dict.keys())]
            ts_sub = [getattr(pecaso_dict[product].two_stage_suboptimal_batch, metric)/\
                       max(pecaso_dict[product].production_envelope.yield_ub)*100 
                       if pecaso_dict[product].two_stage_suboptimal_batch else np.nan
                       for  product in list(pecaso_dict.keys())]  

        data_dict = {'ts_best': ts_best,
                     'os_best': os_best,
                     'ts_sub': ts_sub}
        process_metrics[objective][metric] = pd.DataFrame(data=data_dict, index=products)
        process_metrics[objective][metric] = process_metrics[objective][metric].fillna(0)
        process_metrics[objective][metric]['carbon_number'] = process_metrics[objective][metric].index.map(carbon_dict)
        process_metrics[objective][metric]['product'] = process_metrics[objective][metric].index
        process_metrics[objective][metric] = process_metrics[objective][metric].sort_values(by=['carbon_number','product'],
                                                                                            ascending=[False,False])
        trace_list = []
        process_metrics[objective][metric] = process_metrics[objective][metric].rename(index=metabolite_names_dict)
        carbon_numbers = ["1", "2", "3", "4", "5", "6", ">6"]
        process_metrics[objective][metric] = process_metrics[objective][metric].reindex((metabolite_order))

In [47]:
def plot_process_metrics(process_metrics, ranges, tickvals, filename=None):
    units = [' (g/L.h)', ' (% Max Theoretical Yield)', ' (g/L)']
    metrics = ['batch_productivity','batch_yield','batch_titer']
    titles = ['Productivity', 'Yield', 'Titer']
    for i, metric in enumerate(metrics):
        trace_list = []
        metric_df = process_metrics[metric]
        if metric in ['batch_yield']:
            ts_sub_values = [value if (value >= ranges[i][0] or np.isnan(value)) else ranges[i][0] 
                             for value in metric_df['ts_sub'].values]
            os_best_values = [value if (value >= ranges[i][0] or np.isnan(value)) else ranges[i][0] 
                              for value in metric_df['os_best'].values]
            ts_best_values = [value if (value >= ranges[i][0] or np.isnan(value)) else ranges[i][0] 
                              for value in metric_df['ts_best'].values]
        else:
            ts_sub_values = [value if (value >= 10**ranges[i][0] or np.isnan(value)) else 10**ranges[i][0] 
                 for value in metric_df['ts_sub'].values]
            os_best_values = [value if (value >= 10**ranges[i][0] or np.isnan(value)) else 10**ranges[i][0] 
                              for value in metric_df['os_best'].values]
            ts_best_values = [value if (value >= 10**ranges[i][0] or np.isnan(value)) else 10**ranges[i][0] 
                              for value in metric_df['ts_best'].values]

        trace_list.append(go.Scatter(x=list(metric_df.index),
                                     y=ts_sub_values,
                                     mode='markers', 
                                     marker=dict(color='#FF5555',
                                                 size=10),
                                     name='Traditional Two Stage Process',
                                     legendgroup='ts_sub',
                                     showlegend=True))

        trace_list.append(go.Scatter(x=list(metric_df.index),
                                     y=os_best_values,
                                     mode='markers',
                                     marker=dict(color='#76D1F4',
                                                 size=10),
                                     name='Best One Stage Process',
                                     legendgroup='os_best',
                                     showlegend=True))

        trace_list.append(go.Scatter(x=list(metric_df.index), 
                                     y=ts_best_values,
                                     mode='markers',
                                     marker=dict(color='#8DC447',
                                                 size=10),
                                     name='Best Two Stage Process',
                                     legendgroup='ts_best',
                                     showlegend=True))

        layout = go.Layout(height=400,
                           width=992,
                           showlegend=True,
                           legend=dict(orientation='h',
                                       x=0,
                                       y=-0.6),
                           hovermode='closest',

                           yaxis=dict(title=titles[i]+units[i], title_standoff=0,
                                      titlefont=dict(family='Arial', size=14, color='black'),
                                      type='log' if metric!='batch_yield' else 'linear',
                                      showline=True, linewidth=1.5, linecolor='black',
                                      mirror=True,
                                      ticks='outside',
                                      ticklen=4, tickcolor='black',
                                      nticks=12,
                                      tickvals=tickvals[i],
                                      tickfont=dict(size=12, color='black'),
                                      side='top',
                                      showgrid=True,
                                      gridcolor='#707070', gridwidth=0.5,
                                      range=ranges[i]
                                     ),

                           xaxis=dict(type='category',
                                      showline=True, linewidth=1.5, linecolor='black',
                                      ticks='outside',
                                      ticklen=4,
                                      tickangle=-90,
                                      tickfont=dict(size=9.25,color='black',family='Arial'),
                                      range=[-1,70],
                                      gridcolor='#707070',
                                      gridwidth=0.5,
                                      mirror=True))

        fig = go.Figure(data=trace_list,layout=layout)


        plot(fig)

        if filename:
            if not os.path.exists('images'):
                os.mkdir('images')
            pio.write_image(fig, 'images/process_metrics_'+str(titles[i]).lower()+'_'+
                            filename+'.svg')


# Objective: Productivity #

# Constraints: None #

In [48]:
tickvals = [[0.2,0.4,0.6,0.8,1,2,4,6,8,10,20],
            [20,30,40,50,60,70,80],
            [2, 4, 6, 8, 10, 20, 40, 60, 80, 100, 200]]

ranges = [[-0.6, 1.2],
          [25, 65],
          [0.6, 2.2]]

plot_process_metrics(process_metrics['prod'], ranges, tickvals)#, filename='objA')

# Objective: Productivity #

# Constraints: 75% Max Yield #

In [49]:
tickvals = [[0.2,0.4,0.6,0.8,1,2,4,6,8,10,20],
            [20,30,40,50,60,70,80,90,100],
            [2, 4, 6, 8, 10, 20, 40, 60, 80, 100, 200]]

ranges = [[-0.9, 1.05],
          [40, 100],
          [1, 2.2]]

plot_process_metrics(process_metrics['prod_75_yield'], ranges, tickvals)#, filename='objB')

# Objective: Yield #

# Constraints: 2 g/L.h Productivity#

In [50]:
tickvals = [[0.2,0.4,0.6,0.8,1,2,4,6,8,10,20],
            [20,30,40,50,60,70,80,90,100],
            [2, 4, 6, 8, 10, 20, 40, 60, 80, 100, 200]]

ranges = [[-0, 0.6020],
          [50, 90],
          [1, 2.2]]

plot_process_metrics(process_metrics['yield_2_prod'], ranges, tickvals)#, filename='objC')